In [ ]:
!pip install cffi==1.15.1
!pip install mip
!pip install gurobipy

import numpy as np
import pandas as pd
from mip import *
import gurobipy as gp
from gurobipy import GRB

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import plotly.express as px
import plotly.graph_objects as go

np.random.seed(42)



# Importando Dados

In [ ]:
df=pd.read_csv('Pasta(Planilha1).csv')
df_backup=df.copy()
df['destination'] = df['destination'].replace(2, 0)# Substituindo o destino pilha para 0
df['horas'] = np.random.uniform(8, 21, size=len(df))#Criando coluna horas aleatoria seguindo uma distribuição normal
df['ano'] =0
df_backup=df.copy()


# Definindo Precedencia de Blocos


In [ ]:
precedence=df.copy()
precedence = precedence.drop(columns=['tonn','blockvalue','destination','CU%','process_profit','Profit','horas','ano'])

coords_to_id = {(row['x'], row['y'], row['z']): row['id'] for _, row in precedence.iterrows()}

In [ ]:
precedence['prec1'] = list(zip(precedence['x'], precedence['y'], precedence['z'] + 1))
precedence['prec2'] = list(zip(precedence['x']-1, precedence['y']+1, precedence['z'] + 1))
precedence['prec3'] = list(zip(precedence['x']-1, precedence['y'], precedence['z'] + 1))
precedence['prec4'] = list(zip(precedence['x']-1, precedence['y']-1, precedence['z'] + 1))
precedence['prec5'] = list(zip(precedence['x'], precedence['y']+1, precedence['z'] + 1))
precedence['prec6'] = list(zip(precedence['x'], precedence['y']-1, precedence['z'] + 1))
precedence['prec7'] = list(zip(precedence['x']+1, precedence['y']+1, precedence['z'] + 1))
precedence['prec8'] = list(zip(precedence['x']+1, precedence['y'], precedence['z'] + 1))
precedence['prec9'] = list(zip(precedence['x']+1, precedence['y']-1, precedence['z'] + 1))


In [ ]:
def replace_tuple(tuple_value):
    return coords_to_id.get(tuple_value, -1)

# Aplica a função em todas as colunas 'prec' do DataFrame
for col in precedence.columns:
    if col.startswith('prec'):
        precedence[col] = precedence[col].apply(replace_tuple).astype(int)

precedence_backup=precedence.copy()



In [ ]:
def apagar_prec(dataframe, lista_numeros):
    # Filtra as colunas que começam com 'prec'
    colunas_prec = [col for col in dataframe.columns if col.startswith('prec')]

    # Substitui os valores nas colunas filtradas de acordo com a lista de números
    for coluna in colunas_prec:
        dataframe[coluna] = dataframe[coluna].apply(lambda x: -1 if x in lista_numeros else x)

    return dataframe

# Solver Gurobi


In [ ]:
params = {
"WLSACCESSID": '9f9c2245-22ce-4d25-9def-efb0a12c39d6',
"WLSSECRET": '42d0ad98-0a28-4682-99c2-e0346c355ed9',
"LICENSEID":2519005,
'OutputFlag': 0
}
env = gp.Env(params=params)



In [ ]:
lucro_anual = []

# Loop pelos anos
while True:
    block_id = list(df['id'].values)
    destination = list(df['destination'].values)
    profit = list(df['Profit'].values)
    tonn = list(df['tonn'].values)
    horas = list(df['horas'].values)

    # Modelo
    mine_model = gp.Model(env=env)

    # Definindo variáveis
    selected = mine_model.addVars(block_id, vtype=GRB.BINARY, name="selected")

    # Definindo a função objetivo
    mine_model.setObjective(gp.quicksum(selected[i] * profit[i] for i in block_id), GRB.MAXIMIZE)

    # Restrições de precedência
    for index, row in precedence.iterrows():
        for col in precedence.columns:
            if col.startswith('prec'):
                if row[col] != -1:
                    mine_model.addConstr(selected[row[col]] >= selected[row['id']])

    # Outras restrições
    mine_model.addConstr(gp.quicksum(destination[i] * horas[i] * selected[i] for i in block_id) <= 7884)
    mine_model.addConstr(gp.quicksum(destination[i] * tonn[i] * selected[i] for i in block_id) <= 10000000)
    mine_model.addConstr(gp.quicksum(horas[i] * selected[i] for i in block_id) <= 80000000)

    # Otimizar o modelo
    mine_model.optimize()

    if mine_model.status == GRB.OPTIMAL and mine_model.ObjVal > 0:
        print("Valor da função objetivo:", mine_model.ObjVal)
        lucro_anual.append(mine_model.ObjVal)

        removidos = []
        for index, i in enumerate(block_id):
            if selected[i].x > 0.6:  # Verifica se o bloco está selecionado
                df.at[index, 'Profit'] = 0  # Define o lucro para 0
                df.at[index, 'horas'] = 99999
                df.at[index, 'tonn'] = 100000000
                removidos.append(df.at[index, 'id'])
                if df.at[index, 'ano'] == 0:
                    df.at[index, 'ano'] = k + 1  # Define o ano como k + 1

        precedence = apagar_prec(precedence, removidos)
    else:
        break

    mine_model.reset()

In [ ]:
lucro_anual

In [ ]:
vpl = 0
for t in range(len(lucro_anual)):
    vpl += lucro_anual[t] / ((1 + 0.15) ** t)
print(vpl)

## Visualizando Sequenciamento

In [ ]:
df = df[df['ano'] != 0]

unique_years = df['ano'].unique()

fig = go.Figure(data=[go.Scatter3d(
    x=df['x'],
    y=df['y'],
    z=df['z'],
    mode='markers',
    marker=dict(
        size=5,
        symbol='square',
        color=df['ano'],
        colorscale='Plasma',
        colorbar=dict(
            title='Ano de Processamento',
            tickvals=unique_years,
            ticktext=unique_years
        )
    ),
    text=['Ano: ' + str(year) for year in df['ano']]
)])

fig.update_layout(
    scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z'
    ),
    title='Blocos Processados por Ano'
)

fig.show()

## Histograma

In [ ]:
merged_df = pd.merge(df, df_backup, on='id')



In [ ]:
df_processados = merged_df.loc[(merged_df['ano_x'] > 0) & (merged_df['destination_x'] == 1)]

soma_por_ano_processados = df_processados.groupby('ano_x')['tonn_y'].sum()


df_extraidos = merged_df.loc[(merged_df['ano_x'] > 0) ]

soma_por_ano_extraidos = df_extraidos.groupby('ano_x')['tonn_y'].sum()

anos = sorted(set(soma_por_ano_processados.index) | set(soma_por_ano_extraidos.index))

toneladas_processadas = [soma_por_ano_processados.get(ano, 0) for ano in anos]
toneladas_extraidas = [soma_por_ano_extraidos.get(ano, 0) for ano in anos]

largura_barra = 0.35

posicoes = range(len(anos))

plt.bar(posicoes, toneladas_processadas, largura_barra, label='Toneladas Processadas na Usina')

plt.bar([posicao + largura_barra for posicao in posicoes], toneladas_extraidas, largura_barra, color='orange', label='Toneladas Extraídas na Mina')

plt.xlabel('Ano')
plt.ylabel('Toneladas')


plt.xticks([posicao + largura_barra / 2 for posicao in posicoes], anos)

plt.ticklabel_format(style='plain', axis='y')

plt.title('Soma das Toneladas Processadas e Extraídas por Ano')

plt.legend()

plt.show()


In [ ]:
df_processados = merged_df.loc[(merged_df['ano_x'] > 0) & (merged_df['destination_x'] == 1)]
soma_por_ano_processados = df_processados.groupby('ano_x')['tonn_y'].sum()/1000000
soma_hora_processados = df_processados.groupby('ano_x')['horas_y'].sum()

df_extraidos = merged_df.loc[(merged_df['ano_x'] > 0)]
soma_por_ano_extraidos = df_extraidos.groupby('ano_x')['tonn_y'].sum()/1000000

anos = sorted(set(soma_por_ano_processados.index) | set(soma_por_ano_extraidos.index))
toneladas_processadas = [soma_por_ano_processados.get(ano, 0) for ano in anos]
toneladas_extraidas = [soma_por_ano_extraidos.get(ano, 0) for ano in anos]
horas_processadas = [soma_hora_processados.get(ano, 0) for ano in anos]

largura_barra = 0.35
posicoes = range(len(anos))

fig, ax1 = plt.subplots(figsize=(12, 6))

# Gráfico de barras para toneladas processadas e extraídas
barras1 = ax1.bar(posicoes, toneladas_processadas, largura_barra, label='Toneladas Processadas na Usina')
barras2 = ax1.bar([posicao + largura_barra for posicao in posicoes], toneladas_extraidas, largura_barra, color='orange', label='Toneladas Extraídas na Mina')

# Configurações do eixo x e y para o primeiro gráfico
ax1.set_xlabel('Ano')
ax1.set_ylabel('Milhões de Toneladas')
ax1.set_xticks([posicao + largura_barra / 2 for posicao in posicoes])
ax1.set_xticklabels(anos)
ax1.ticklabel_format(style='plain', axis='y')
ax1.legend(loc='upper left')

# Eixo y secundário para horas processadas
ax2 = ax1.twinx()
barras3 = ax2.plot([posicao + 0.2 for posicao in posicoes], horas_processadas, color='green', marker='o', linestyle='-', label='Horas Processadas')
ax2.set_ylabel('Horas Processadas')
ax2.set_ylim(0, 15000)

ax2.legend(loc='upper right')

plt.title('Soma das Toneladas Processadas e Extraídas por Ano e Horas Processadas')
fig.tight_layout()
plt.show()

# Solver CBC

In [ ]:
df=df_backup.copy()
precedence=precedence_backup.copy()


In [ ]:
lucro_anual=[]
while True:
    block_id = list(df['id'].values)
    destination = list(df['destination'].values)
    profit = list(df['Profit'].values)
    tonn=list(df['tonn'].values)
    horas = list(df['horas'].values)
    # Modelo
    mine_model = Model(sense = MAXIMIZE, solver_name=CBC)
    mine_model.max_mip_gap=0.05
    # Variables
    selected = [mine_model.add_var(var_type=BINARY) for i in block_id]
    # Objective
    mine_model.objective = maximize(xsum( selected[i] * profit[i]for i in block_id))
    # Restrições de precedência
    for index, row in precedence.iterrows():
        for col in precedence.columns:
            if col.startswith('prec'):
                if row[col] != -1:
                    mine_model += selected[row[col]] >= selected[row['id']]
    mine_model += xsum(destination[i] * horas[i] * selected[i] for i in block_id) <= 7884
    mine_model += xsum(destination[i] *tonn[i] * selected[i]for i in block_id) <= 10000000
    mine_model += xsum(tonn[i] * selected[i] for i in block_id) <= 80000000

    mine_model.optimize()

    if mine_model.status == OptimizationStatus.OPTIMAL and mine_model.objective_value>0:
        print("Valor da função objetivo:", mine_model.objective_value)
        lucro_anual.append(mine_model.objective_value)

        removidos=[]
        for index, value in enumerate(selected):
            #Altera valores dos blocos para facilitar o modelo decidir blocos selecionados nas próximas iterações aumentando custos das restrições e diminuindo
            #o valor objetivo dos blocos
            if value.x >0.6:  # Verifica se o bloco está selecionado
                df.at[index, 'Profit'] = 0  # Define o lucro para 0
                df.at[index, 'horas'] = 99999
                df.at[index, 'tonn'] = 100000000
                removidos.append( df.at[index, 'id'])

                if df.at[index, 'ano']==0:
                    df.at[index, 'ano'] = k + 1  # Define o ano como k + 1

        precedence=apagar_prec(precedence,removidos)
    else:break

    mine_model.clear()



In [ ]:
vpl = 0
for t in range(len(lucro_anual)):
    vpl += lucro_anual[t] / ((1 + 0.15) ** t)
print(vpl)

## Visualizando Sequenciamento

In [ ]:
df = df[df['ano'] != 0]

fig = go.Figure(data=[go.Scatter3d(
    x=df['x'],
    y=df['y'],
    z=df['z'],
    mode='markers',
    marker=dict(
        size=5,
        color=df['ano'],
        colorscale='Plasma',
        colorbar=dict(
            title='Ano de Processamento',
            tickvals=unique_years,
            ticktext=unique_years
        )
    ),
    text=['Ano: ' + str(year) for year in df['ano']]
)])

fig.update_layout(
    scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z'
    ),
    title='Blocos Processados por Ano'
)

fig.show()


## Histogramas

In [ ]:
merged_df = pd.merge(df, df_backup, on='id')



In [ ]:
df_processados = merged_df.loc[(merged_df['ano_x'] > 0) & (merged_df['destination_x'] == 1)]

soma_por_ano_processados = df_processados.groupby('ano_x')['tonn_y'].sum()

df_extraidos = merged_df.loc[(merged_df['ano_x'] > 0) ]

soma_por_ano_extraidos = df_extraidos.groupby('ano_x')['tonn_y'].sum()

anos = sorted(set(soma_por_ano_processados.index) | set(soma_por_ano_extraidos.index))

toneladas_processadas = [soma_por_ano_processados.get(ano, 0) for ano in anos]
toneladas_extraidas = [soma_por_ano_extraidos.get(ano, 0) for ano in anos]

largura_barra = 0.35

posicoes = range(len(anos))

plt.bar(posicoes, toneladas_processadas, largura_barra, label='Toneladas Processadas na Usina')

plt.bar([posicao + largura_barra for posicao in posicoes], toneladas_extraidas, largura_barra, color='orange', label='Toneladas Extraídas na Mina')

plt.xlabel('Ano')
plt.ylabel('Toneladas')


plt.xticks([posicao + largura_barra / 2 for posicao in posicoes], anos)

plt.ticklabel_format(style='plain', axis='y')

plt.title('Soma das Toneladas Processadas e Extraídas por Ano')

plt.legend()

plt.show()


In [ ]:
df_processados = merged_df.loc[(merged_df['ano_x'] > 0) & (merged_df['destination_x'] == 1)]
soma_por_ano_processados = df_processados.groupby('ano_x')['tonn_y'].sum()/1000000
soma_hora_processados = df_processados.groupby('ano_x')['horas_y'].sum()

df_extraidos = merged_df.loc[(merged_df['ano_x'] > 0)]
soma_por_ano_extraidos = df_extraidos.groupby('ano_x')['tonn_y'].sum()/1000000

anos = sorted(set(soma_por_ano_processados.index) | set(soma_por_ano_extraidos.index))
toneladas_processadas = [soma_por_ano_processados.get(ano, 0) for ano in anos]
toneladas_extraidas = [soma_por_ano_extraidos.get(ano, 0) for ano in anos]
horas_processadas = [soma_hora_processados.get(ano, 0) for ano in anos]

largura_barra = 0.35
posicoes = range(len(anos))

fig, ax1 = plt.subplots(figsize=(12, 6))

# Gráfico de barras para toneladas processadas e extraídas
barras1 = ax1.bar(posicoes, toneladas_processadas, largura_barra, label='Toneladas Processadas na Usina')
barras2 = ax1.bar([posicao + largura_barra for posicao in posicoes], toneladas_extraidas, largura_barra, color='orange', label='Toneladas Extraídas na Mina')

# Configurações do eixo x e y para o primeiro gráfico
ax1.set_xlabel('Ano')
ax1.set_ylabel('Toneladas')
ax1.set_xticks([posicao + largura_barra / 2 for posicao in posicoes])
ax1.set_xticklabels(anos)
ax1.ticklabel_format(style='plain', axis='y')
ax1.legend(loc='upper left')

# Eixo y secundário para horas processadas
ax2 = ax1.twinx()
barras3 = ax2.plot([posicao + 0.2 for posicao in posicoes], horas_processadas, color='green', marker='o', linestyle='-', label='Horas Processadas')
ax2.set_ylabel('Horas Processadas')
ax2.set_ylim(0, 15000)

ax2.legend(loc='upper right')

plt.title('Soma das Toneladas Processadas e Extraídas por Ano e Horas Processadas')
fig.tight_layout()
plt.show()

# Ano a Ano

In [ ]:
unique_years = np.sort(df['ano'].unique())

# Criar uma colormap categórica usando ListedColormap
num_colors = len(unique_years)
cmap = plt.get_cmap('plasma', num_colors)

# Criar um subplot para cada ano
num_years = len(unique_years)
fig = plt.figure(figsize=(15, 5 * num_years))

for i, year in enumerate(unique_years):
    ax = fig.add_subplot(num_years, 1, i + 1, projection='3d')

    # Filtrar o DataFrame para o ano atual
    df_year = df[df['ano'] == year]

    # Mapear ano para índice de cor
    color_index = np.where(unique_years == year)[0][0]

    # Plotar os pontos 3D com a cor correspondente ao ano de processamento
    sc = ax.scatter(df_year['x'], df_year['y'], df_year['z'], c=[cmap(color_index)] * len(df_year))

    # Adicionar rótulos aos eixos
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.set_title(f'Blocos Processados no Ano {year}')



# Ajustar o layout
plt.tight_layout()
plt.show()

# HIGHS SOLVER

In [ ]:
!pip install pyomo
!pip install highspy

from pyomo.environ import *
from pyomo.contrib.appsi.solvers.highs import Highs

In [ ]:
df=df_backup.copy()
precedence=precedence_backup.copy()
anos = df['tonn'].sum() / 10000000
anos=int(anos)+1
anos

23

In [ ]:
lucro_anual = []

for k in range(anos):
    block_id = list(df['id'].values)
    destination = list(df['destination'].values)
    profit = list(df['Profit'].values)
    tonn = list(df['tonn'].values)
    horas = list(df['horas'].values)

    # Modelo
    model = ConcreteModel()

    # Conjuntos
    model.blocks = Set(initialize=block_id)

    # Parâmetros
    model.profit = Param(model.blocks, initialize={block_id[i]: profit[i] for i in range(len(block_id))})
    model.destination = Param(model.blocks, initialize={block_id[i]: destination[i] for i in range(len(block_id))})
    model.tonn = Param(model.blocks, initialize={block_id[i]: tonn[i] for i in range(len(block_id))})
    model.horas = Param(model.blocks, initialize={block_id[i]: horas[i] for i in range(len(block_id))})

    # Variáveis
    model.selected = Var(model.blocks, within=Binary)

    # Função objetivo
    def objective_rule(model):
        return sum(model.selected[i] * model.profit[i] for i in model.blocks)
    model.objective = Objective(rule=objective_rule, sense=maximize)

    # Restrições de precedência
    def precedence_rule(model, i, j):
        return model.selected[j] >= model.selected[i]

    for index, row in precedence.iterrows():
        for col in precedence.columns:
            if col.startswith('prec'):
                if row[col] != -1:
                    model.add_component(f'precedence_{index}_{col}', Constraint(expr=precedence_rule(model, row['id'], row[col])))

    # Outras restrições
    model.total_hours = Constraint(expr=sum(model.destination[i] * model.horas[i] * model.selected[i] for i in model.blocks) <= 7884)
    model.total_tonn = Constraint(expr=sum(model.destination[i] * model.tonn[i] * model.selected[i] for i in model.blocks) <= 10000000)
    model.max_hours = Constraint(expr=sum(model.horas[i] * model.selected[i] for i in model.blocks) <= 80000000)

    # Resolver o modelo usando HIGHS
    solver = SolverFactory('highs')
    results = solver.solve(model, tee=True)

    if (results.solver.status == SolverStatus.ok) and (results.solver.termination_condition == TerminationCondition.optimal):
        obj_value = model.objective()
        print("Valor da função objetivo:", obj_value)
        lucro_anual.append(obj_value)

        removidos = []
        for index, i in enumerate(block_id):
            if model.selected[i].value > 0.6:  # Verifica se o bloco está selecionado
                df.at[index, 'Profit'] = 0  # Define o lucro para 0
                df.at[index, 'horas'] = 99999
                df.at[index, 'tonn'] = 100000000
                removidos.append(df.at[index, 'id'])
                if df.at[index, 'ano'] == 0:
                    df.at[index, 'ano'] = k + 1  # Define o ano como k + 1

        precedence = apagar_prec(precedence, removidos)
    else:
        break

    model = ConcreteModel()  # Reset do modelo

Failed to set executable for solver asl. File with name=highs either does not exist or it is not executable. To skip this validation, call set_executable with validate=False.
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pyomo/opt/base/solvers.py", line 162, in __call__
    opt = self._cls[_implicit_solvers[mode]](**kwds)
  File "/usr/local/lib/python3.10/dist-packages/pyomo/solvers/plugins/solvers/ASL.py", line 46, in __init__
    SystemCallSolver.__init__(self, **kwds)
  File "/usr/local/lib/python3.10/dist-packages/pyomo/opt/solver/shellcmd.py", line 66, in __init__
    self.set_executable(name=executable, validate=validate)
  File "/usr/local/lib/python3.10/dist-packages/pyomo/opt/solver/shellcmd.py", line 115, in set_executable
    raise ValueError(
ValueError: Failed to set executable for solver asl. File with name=highs either does not exist or it is not executable. To skip this validation, call set_executable with validate=False.


RuntimeError: Attempting to use an unavailable solver.

The SolverFactory was unable to create the solver "highs"
and returned an UnknownSolver object.  This error is raised at the point
where the UnknownSolver object was used as if it were valid (by calling
method "solve").

The original solver was created with the following parameters:
	executable: highs
	type: highs
	_args: ()
	options: {}